In [1]:
%load_ext lab_black

In [21]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl

# quality of the image
mpl.rcParams["figure.dpi"] = 150

In [22]:
# data
client_tr = pd.read_csv("../data/client.train.csv", parse_dates=["birthdate"])
client_te = pd.read_csv("../data/client.test.csv", parse_dates=["birthdate"])
client_ch = pd.read_csv("../data/client.challenge.csv", parse_dates=["birthdate"])
orders = pd.read_csv("../data/orders.csv", parse_dates=["datetime"])
product = pd.read_csv("../data/product.csv")

product

,product_id,category_lv0,category_lv1,category_lv2,category_lv3,category_lv4
0,px01237,mk38,mk38:072,mk38:072:149,mk38:072:149:218,mk38:072:149:218:04846
1,px11510,mk38,mk38:072,mk38:072:149,mk38:072:149:218,mk38:072:149:218:01337
2,px03782,mk38,mk38:072,mk38:072:019,mk38:072:019:066,mk38:072:019:066:02321
3,px04092,mk38,mk38:072,mk38:072:019,mk38:072:019:054,mk38:072:019:054:06035
4,px07429,mk38,mk38:072,mk38:072:078,mk38:072:078:182,mk38:072:078:182:07218
...,...,...,...,...,...,...
12654,px11965,mk35,mk35:072,mk35:072:122,mk35:072:122:182,mk35:072:122:182:00392
12655,px05192,mk35,mk35:072,mk35:072:122,mk35:072:122:182,mk35:072:122:182:02028
12656,px05391,mk38,mk38:107,mk38:107:357,mk38:107:357:182,mk38:107:357:182:08006
12657,px00861,mk35,mk35:072,mk35:072:177,mk35:072:177:182,mk35:072:177:182:03018


In [23]:
client_tr["client_id"].unique().size

23292

In [24]:
client_tr

,client_id,birthdate,state,gender,is_churn
0,cx05072,1972-03-22,st25,cg2,False
1,cx03836,1966-08-14,st15,cg1,False
2,cx08472,1954-08-07,st15,cg1,False
3,cx12946,1955-11-16,st15,cg1,False
4,cx19045,1958-10-25,st15,cg2,False
...,...,...,...,...,...
23287,cx05312,1981-08-29,st18,cg1,False
23288,cx02434,1997-03-15,st04,cg2,False
23289,cx16024,1950-10-09,st11,cg1,False
23290,cx11285,1976-07-03,st11,cg2,False


In [25]:
orders

,order_id,client_id,product_id,datetime,units,price,payment_method,installments,delivery_days,delivery_state,device,source
0,ox008433,cx05072,px04611,2019-10-07 15:20:00,1.0,49.90,pm05,NaN,NaN,st07,dv9,sc08
1,ox070389,cx05072,px12305,2019-10-11 17:42:00,1.0,69.90,pm05,NaN,NaN,st07,dv9,sc08
2,ox061137,cx06840,px07033,2019-01-19 13:09:00,1.0,35.90,pm05,NaN,NaN,st07,dv9,sc08
3,ox061137,cx06840,px05992,2019-01-19 13:09:00,3.0,35.90,pm05,NaN,NaN,st07,dv9,sc08
4,ox038612,cx06840,px10040,2019-04-18 14:36:00,1.0,39.00,pm05,NaN,NaN,st07,dv9,sc08
...,...,...,...,...,...,...,...,...,...,...,...,...
168467,ox081422,cx15796,px08541,2019-06-25 11:50:00,1.0,246.50,pm14,10.0,21.0,st01,dv5,sc11
168468,ox081422,cx15796,px05166,2019-06-25 11:50:00,1.0,652.50,pm14,10.0,21.0,st01,dv5,sc11
168469,ox128219,cx15796,px12043,2019-09-06 11:45:00,2.0,15.00,pm05,NaN,NaN,st07,dv9,sc08
168470,ox126589,cx15796,px12043,2019-09-07 04:39:00,1.0,15.00,pm10,1.0,0.0,st01,dv3,sc12


In [26]:
# join orders + product
orders_product_df = pd.merge(left=orders, right=product, on="product_id", how="left")

In [27]:
orders_product_df.columns

Index(['order_id', 'client_id', 'product_id', 'datetime', 'units', 'price',
       'payment_method', 'installments', 'delivery_days', 'delivery_state',
       'device', 'source', 'category_lv0', 'category_lv1', 'category_lv2',
       'category_lv3', 'category_lv4'],
      dtype='object')

In [28]:
orders_product_df[orders_product_df["client_id"] == "cx05072"]

,order_id,client_id,product_id,datetime,units,price,payment_method,installments,delivery_days,delivery_state,device,source,category_lv0,category_lv1,category_lv2,category_lv3,category_lv4
0,ox008433,cx05072,px04611,2019-10-07 15:20:00,1.0,49.9,pm05,NaN,NaN,st07,dv9,sc08,mk04,mk04:128,mk04:128:103,mk04:128:103:103,mk04:128:103:103:03294
1,ox070389,cx05072,px12305,2019-10-11 17:42:00,1.0,69.9,pm05,NaN,NaN,st07,dv9,sc08,mk04,mk04:292,mk04:292:246,mk04:292:246:103,mk04:292:246:103:04338


In [29]:
product

,product_id,category_lv0,category_lv1,category_lv2,category_lv3,category_lv4
0,px01237,mk38,mk38:072,mk38:072:149,mk38:072:149:218,mk38:072:149:218:04846
1,px11510,mk38,mk38:072,mk38:072:149,mk38:072:149:218,mk38:072:149:218:01337
2,px03782,mk38,mk38:072,mk38:072:019,mk38:072:019:066,mk38:072:019:066:02321
3,px04092,mk38,mk38:072,mk38:072:019,mk38:072:019:054,mk38:072:019:054:06035
4,px07429,mk38,mk38:072,mk38:072:078,mk38:072:078:182,mk38:072:078:182:07218
...,...,...,...,...,...,...
12654,px11965,mk35,mk35:072,mk35:072:122,mk35:072:122:182,mk35:072:122:182:00392
12655,px05192,mk35,mk35:072,mk35:072:122,mk35:072:122:182,mk35:072:122:182:02028
12656,px05391,mk38,mk38:107,mk38:107:357,mk38:107:357:182,mk38:107:357:182:08006
12657,px00861,mk35,mk35:072,mk35:072:177,mk35:072:177:182,mk35:072:177:182:03018


In [30]:
product.drop_duplicates()
# sem duplicates

,product_id,category_lv0,category_lv1,category_lv2,category_lv3,category_lv4
0,px01237,mk38,mk38:072,mk38:072:149,mk38:072:149:218,mk38:072:149:218:04846
1,px11510,mk38,mk38:072,mk38:072:149,mk38:072:149:218,mk38:072:149:218:01337
2,px03782,mk38,mk38:072,mk38:072:019,mk38:072:019:066,mk38:072:019:066:02321
3,px04092,mk38,mk38:072,mk38:072:019,mk38:072:019:054,mk38:072:019:054:06035
4,px07429,mk38,mk38:072,mk38:072:078,mk38:072:078:182,mk38:072:078:182:07218
...,...,...,...,...,...,...
12654,px11965,mk35,mk35:072,mk35:072:122,mk35:072:122:182,mk35:072:122:182:00392
12655,px05192,mk35,mk35:072,mk35:072:122,mk35:072:122:182,mk35:072:122:182:02028
12656,px05391,mk38,mk38:107,mk38:107:357,mk38:107:357:182,mk38:107:357:182:08006
12657,px00861,mk35,mk35:072,mk35:072:177,mk35:072:177:182,mk35:072:177:182:03018


In [31]:
product.isnull().sum() > 0
# sem missing

product_id      False
category_lv0    False
category_lv1    False
category_lv2    False
category_lv3    False
category_lv4    False
dtype: bool

In [32]:
product[["category_lv0"]].value_counts().reset_index()

,category_lv0,0
0,mk17,1273
1,mk04,1229
2,mk29,902
3,mk22,859
4,mk03,680
5,mk21,627
6,mk16,562
7,mk10,459
8,mk06,402
9,mk25,386


In [33]:
product[["category_lv1"]].value_counts().reset_index()

,category_lv1,0
0,mk21:089,563
1,mk03:092,402
2,mk22:168,321
3,mk22:098,301
4,mk03:291,278
...,...,...
291,mk40:254,1
292,mk09:166,1
293,mk41:285,1
294,mk08:065,1


In [34]:
product[["category_lv2"]].value_counts().reset_index()

,category_lv2,0
0,mk03:092:378,237
1,mk04:227:103,207
2,mk21:089:468,195
3,mk28:015:103,176
4,mk22:098:272,173
...,...,...
888,mk37:045:103,1
889,mk11:032:103,1
890,mk06:147:103,1
891,mk29:068:216,1


In [35]:
product[["category_lv3"]].value_counts().reset_index()

,category_lv3,0
0,mk03:092:378:103,237
1,mk04:227:103:103,207
2,mk21:089:468:103,195
3,mk28:015:103:103,176
4,mk22:098:272:103,173
...,...,...
1213,mk17:118:215:103,1
1214,mk04:187:083:103,1
1215,mk25:219:175:099,1
1216,mk29:241:080:103,1


In [36]:
product[["category_lv4"]].value_counts().reset_index()

,category_lv4,0
0,mk35:072:177:182:09721,10
1,mk07:129:434:103:09405,10
2,mk35:072:177:182:04085,8
3,mk35:072:177:182:05523,8
4,mk42:261:065:103:00704,8
...,...,...
10758,mk12:220:106:135:02636,1
10759,mk12:220:106:135:02672,1
10760,mk12:220:106:135:02691,1
10761,mk12:220:106:135:02758,1


In [37]:
orders

,order_id,client_id,product_id,datetime,units,price,payment_method,installments,delivery_days,delivery_state,device,source
0,ox008433,cx05072,px04611,2019-10-07 15:20:00,1.0,49.90,pm05,NaN,NaN,st07,dv9,sc08
1,ox070389,cx05072,px12305,2019-10-11 17:42:00,1.0,69.90,pm05,NaN,NaN,st07,dv9,sc08
2,ox061137,cx06840,px07033,2019-01-19 13:09:00,1.0,35.90,pm05,NaN,NaN,st07,dv9,sc08
3,ox061137,cx06840,px05992,2019-01-19 13:09:00,3.0,35.90,pm05,NaN,NaN,st07,dv9,sc08
4,ox038612,cx06840,px10040,2019-04-18 14:36:00,1.0,39.00,pm05,NaN,NaN,st07,dv9,sc08
...,...,...,...,...,...,...,...,...,...,...,...,...
168467,ox081422,cx15796,px08541,2019-06-25 11:50:00,1.0,246.50,pm14,10.0,21.0,st01,dv5,sc11
168468,ox081422,cx15796,px05166,2019-06-25 11:50:00,1.0,652.50,pm14,10.0,21.0,st01,dv5,sc11
168469,ox128219,cx15796,px12043,2019-09-06 11:45:00,2.0,15.00,pm05,NaN,NaN,st07,dv9,sc08
168470,ox126589,cx15796,px12043,2019-09-07 04:39:00,1.0,15.00,pm10,1.0,0.0,st01,dv3,sc12


In [38]:
client_tr["client_id"].nunique()

23292

In [39]:
client_tr

,client_id,birthdate,state,gender,is_churn
0,cx05072,1972-03-22,st25,cg2,False
1,cx03836,1966-08-14,st15,cg1,False
2,cx08472,1954-08-07,st15,cg1,False
3,cx12946,1955-11-16,st15,cg1,False
4,cx19045,1958-10-25,st15,cg2,False
...,...,...,...,...,...
23287,cx05312,1981-08-29,st18,cg1,False
23288,cx02434,1997-03-15,st04,cg2,False
23289,cx16024,1950-10-09,st11,cg1,False
23290,cx11285,1976-07-03,st11,cg2,False


In [40]:
client_te["client_id"]

0       cx06840
1       cx01931
2       cx35431
3       cx36233
4       cx13540
         ...   
7759    cx19119
7760    cx21963
7761    cx37195
7762    cx16851
7763    cx15796
Name: client_id, Length: 7764, dtype: object

In [41]:
client_on_test = set(client_te["client_id"].to_list())
client_on_train = set(client_tr["client_id"].to_list())

on_test_not_train = client_on_test - client_on_train
len(on_test_not_train)

7764

In [42]:
on_train_not_test = client_on_train - client_on_test
len(on_train_not_test)

23292

Temos apenas valores únicos em todos os datasets (treino e teste)

In [43]:
orders[orders["client_id"].isin(client_te["client_id"].to_list())]

,order_id,client_id,product_id,datetime,units,price,payment_method,installments,delivery_days,delivery_state,device,source
2,ox061137,cx06840,px07033,2019-01-19 13:09:00,1.0,35.90,pm05,NaN,NaN,st07,dv9,sc08
3,ox061137,cx06840,px05992,2019-01-19 13:09:00,3.0,35.90,pm05,NaN,NaN,st07,dv9,sc08
4,ox038612,cx06840,px10040,2019-04-18 14:36:00,1.0,39.00,pm05,NaN,NaN,st07,dv9,sc08
5,ox050949,cx06840,px06767,2019-05-17 13:40:00,1.0,79.90,pm05,NaN,NaN,st07,dv9,sc08
6,ox128683,cx06840,px08113,2019-09-28 11:58:00,1.0,55.90,pm05,NaN,NaN,st07,dv9,sc08
...,...,...,...,...,...,...,...,...,...,...,...,...
168467,ox081422,cx15796,px08541,2019-06-25 11:50:00,1.0,246.50,pm14,10.0,21.0,st01,dv5,sc11
168468,ox081422,cx15796,px05166,2019-06-25 11:50:00,1.0,652.50,pm14,10.0,21.0,st01,dv5,sc11
168469,ox128219,cx15796,px12043,2019-09-06 11:45:00,2.0,15.00,pm05,NaN,NaN,st07,dv9,sc08
168470,ox126589,cx15796,px12043,2019-09-07 04:39:00,1.0,15.00,pm10,1.0,0.0,st01,dv3,sc12


In [44]:
orders_tr_ordered = orders[
    orders["client_id"].isin(client_tr["client_id"].to_list())
].sort_values(["client_id"])

In [45]:
101005 + 33860

134865

In [46]:
orders.shape

(168472, 12)

Variável price possui valores negativos, esse valores podem indicar algum extorno na compra

In [47]:
orders.describe(include="object")

,order_id,client_id,product_id,payment_method,delivery_state,device,source
count,168472,168472,168472,168472,168472,168472,168472
unique,133209,38821,12535,14,26,9,16
top,ox059501,cx33292,px10485,pm05,st07,dv9,sc08
freq,5,101,1875,131927,97820,97794,97796


In [48]:
orders.describe()

,units,price,installments,delivery_days
count,168472.000000,168472.000000,36545.000000,89642.000000
mean,1.228846,451.384291,4.551293,7.017938
std,1.504511,857.769280,4.704385,6.855334
min,0.000000,-8999.000000,1.000000,0.000000
25%,1.000000,39.900000,1.000000,2.000000
50%,1.000000,162.580000,3.000000,5.000000
75%,1.000000,699.000000,7.000000,9.000000
max,110.000000,25648.900000,24.000000,165.000000


In [49]:
orders[orders["client_id"] == "cx29523"].sort_values(["datetime"])

,order_id,client_id,product_id,datetime,units,price,payment_method,installments,delivery_days,delivery_state,device,source
117816,ox096465,cx29523,px02579,2019-08-08 13:00:00,1.0,8999.0,pm14,4.0,2.0,st15,dv5,sc11
117815,ox096465,cx29523,px02579,2019-08-08 14:37:00,1.0,-8999.0,pm05,NaN,NaN,st15,dv5,sc11
117817,ox029497,cx29523,px10091,2019-08-16 14:37:00,1.0,8999.0,pm14,4.0,2.0,st15,dv5,sc11


In [50]:
client_tr[client_tr["client_id"] == "cx29523"]

,client_id,birthdate,state,gender,is_churn
16207,cx29523,1987-04-16,st15,cg2,False


In [51]:
client_tr

,client_id,birthdate,state,gender,is_churn
0,cx05072,1972-03-22,st25,cg2,False
1,cx03836,1966-08-14,st15,cg1,False
2,cx08472,1954-08-07,st15,cg1,False
3,cx12946,1955-11-16,st15,cg1,False
4,cx19045,1958-10-25,st15,cg2,False
...,...,...,...,...,...
23287,cx05312,1981-08-29,st18,cg1,False
23288,cx02434,1997-03-15,st04,cg2,False
23289,cx16024,1950-10-09,st11,cg1,False
23290,cx11285,1976-07-03,st11,cg2,False
